In [ ]:

#In this notebook, we will take the AI generated data set, we will re engineer it to make it more realistic
#primarly, we will over write the salary and balance, to allign with information we have in professions.xls file
#the output will be a final training data set, which will be used as a reference to train our model
#our goal is to make this as realistic as possible

In [273]:
import pandas as pd
import numpy as np
import string
import random 
import requests
from datetime import datetime


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



#load the ai generated df
ai_traininig_df = pd.read_excel('ai_traininig_df.xlsx')
ai_traininig_df


#load the reference file i.e. the professions 
professions = pd.read_excel('professions.xlsx')
countries = pd.read_excel('../countries.xlsx')



#drop duplicates: do not consider salary and balance when dropping duplicates
cols_subset=['age',
 'gender',
 'country',
 'education_level',
 'education_title',
 'profession',
 'years_experience',
 'working' ]
ai_traininig_df= ai_traininig_df.drop_duplicates(subset= cols_subset, keep='last')

print ('done') 

done


In [274]:
#this will always ensure that professions are there in the profession sheet
valid_professions = set(professions['profession'].to_list())
 
ai_traininig_df = ai_traininig_df[ai_traininig_df['profession'].isin(valid_professions)]

#to make sure there are no professions in the data set and not in the professions list
set (ai_traininig_df.profession.to_list()) - set (professions.profession.to_list())  

set()

In [276]:
#this will also ensure no other countries are there
valid_countries = set(professions['country'].to_list())
ai_traininig_df = ai_traininig_df[ai_traininig_df['country'].isin(valid_countries)]

set (ai_traininig_df.country.to_list()) - set (professions.country.to_list())  

set()

In [277]:
#here to get a list of rare professions:
rare_professions = ai_traininig_df['profession'].value_counts()[ai_traininig_df['profession'].value_counts() <= 5].index.tolist()
rare_professions

[]

In [278]:
##first  we should create two new cols for salary and balance, to convert them into EUR:

ai_traininig_df['exchange_rate'] = 0
ai_traininig_df['salary_eur'] =0
ai_traininig_df['balance_eur'] =  0

#now we loop over all rows, to co

for index, row in ai_traininig_df.iterrows():
    country = row['country']
    profession = row['profession']
    years_experience =  row['years_experience']   
    minimum_survival = countries.query("country==@country")['minimum_survival'].values[0]
    exchange_rate = countries.query("country==@country")['exchange_rate'].values[0]
    working = row['working']
    if years_experience>=5: 
        years_experience_ref='5 and more'
        
    else:
         years_experience_ref = row['years_experience'] 
            
            
    #now get the median salary for the professions file
    if len(professions.query("country==@country and years_experience ==@years_experience_ref and profession==@profession")) <1:
        median_salary=0
        sigma=0
       
         
    else:
            
      median_salary = professions.query("country==@country and years_experience ==@years_experience_ref and profession==@profession")['median_salary'].values[0]
      sigma = professions.query("country==@country and years_experience ==@years_experience_ref and profession==@profession")['sigma'].values[0]

    
     
        # Calculate salary and balance for each row. For salary: sample from a log normal distribution
        
# get the salary from a log normal distribution around the median salary
#here we make an assumption that salaries follow a log normal distribution which is right skewed

 
    expected_salary = np.random.lognormal(mean= np.log(median_salary), sigma=sigma, size=1)
    
    #now to exclude people who are not working, to edjust their salary
    if working ==0 :
        expected_salary = minimum_survival *1.2 # just an assumption
#     
#now balance, which is based on years of experience and monthly saving percentage of the amount left beyond basic spending
#i.e. the minimum_survival
    saving_percentage= 0.3 #how much does someone save of the amount left beyond the minimal survival?
    expected_balance= (expected_salary-minimum_survival) * saving_percentage * years_experience *12
    
    #another assumption: if years_experience==0 then expected_balance = expected_salary
    if years_experience==0: 
        expected_balance = expected_salary
        
#     print(median_salary, expected_salary,saving_percentage,years_experience,minimum_survival, balance )  
 
 
 
#     # Update the DataFrame with calculated salary and balance
    ai_traininig_df.at[index, 'salary'] = int (expected_salary)
    ai_traininig_df.at[index, 'balance'] =int  (expected_balance)
    ai_traininig_df.at[index, 'exchange_rate'] = exchange_rate
 



print("done")

done


In [279]:

ai_traininig_df['salary_eur'] =   ai_traininig_df['salary'] / ai_traininig_df['exchange_rate'] 
ai_traininig_df['balance_eur'] =  ai_traininig_df['balance'] /  ai_traininig_df['exchange_rate'] 


In [200]:
ai_traininig_df.query("working==0")

,age,gender,country,education_level,education_title,profession,years_experience,working,salary,balance,hobbies,spendingBehaviour,currency
63,60,female,Spain,Secondary,High School Diploma,Retired Teacher,35,0,1905,40017.0,"['Gardening', 'Walking']",['Low-Income_Necessity Spender'],EUR
97,65,male,Belgium,Postgraduate,Master of Engineering,Retired Civil Engineer,40,0,2064,49536.0,"['Traveling', 'Fishing']",['Middle-Class_Value Seeker'],EUR
200,60,male,Greece,Postgraduate,PhD in Philosophy,Retired Professor,30,0,1416,25488.0,"['Writing', 'Fishing']",['Low-Income_Necessity Spender'],EUR
782,28,male,Portugal,Undergraduate,Bachelor of Tourism,Travel Consultant,4,0,1716,4118.0,"['Traveling', 'Photography']",Low-Income_Necessity Spender,EUR
853,28,female,Portugal,Undergraduate,Bachelor of Tourism,Travel Consultant,4,0,1716,4118.0,"['Traveling', 'Photography']",Low-Income_Necessity Spender,EUR
1164,23,female,Hungary,Undergraduate,Bachelor of Arts in Design,Interior Designer,0,0,469200,469200.0,"['Football', 'Traveling', 'Reading']",['Low-Income_Necessity Spender'],HUF
1182,25,male,Belgium,Undergraduate,Bachelor of Graphic Design,Graphic Designer,0,0,2064,2064.0,['Drawing'],['Low-Income_Necessity Spender'],EUR
1260,23,female,Luxembourg,Undergraduate,Bachelor of Computer Science,IT Support Specialist,0,0,3060,3060.0,"['Drawing', 'Dancing', 'Skating']",['Low-Income_Necessity Spender'],EUR
1277,25,female,Latvia,Postgraduate,Master of Psychology,Clinical Psychologist,0,0,1230,1230.0,"['Swimming', 'Tennis']",['Low-Income_Necessity Spender'],EUR
1310,23,male,Portugal,Undergraduate,Bachelor of Information Technology,IT Specialist,0,0,1716,1716.0,['Photography'],['Low-Income_Necessity Spender'],EUR


In [284]:
len(ai_traininig_df)

1746

In [281]:
ai_traininig_df

,age,gender,country,education_level,education_title,profession,years_experience,working,salary,balance,hobbies,spendingBehaviour,currency,exchange_rate,salary_eur,balance_eur
0,28,male,Germany,Undergraduate,Bachelor of Engineering,Mechanical Engineer,4,1,5930,58117.0,"['Cycling', 'Reading', 'Gaming']",['Middle-Class_Value Seeker'],EUR,1.00,5930.000000,58117.000000
1,31,female,Poland,Postgraduate,Master of Architecture,Architect,6,1,10826,102098.0,"['Yoga', 'Traveling', 'Photography']",['Middle-Class_Value Seeker'],PLN,4.50,2405.777778,22688.444444
2,37,male,France,Postgraduate,Master of Finance,Investment Banker,12,1,5031,130550.0,"['Golf', 'Traveling', 'Fine Dining']",['Wealthy_Status_Spender'],EUR,1.00,5031.000000,130550.000000
3,29,female,Italy,Undergraduate,Bachelor of Medicine,Doctor,5,1,4285,48610.0,"['Painting', 'Running', 'Music']",['Middle-Class_Value Seeker'],EUR,1.00,4285.000000,48610.000000
4,40,male,Sweden,Undergraduate,Bachelor of Social Work,Social Worker,14,1,51446,1532006.0,"['Reading', 'Swimming']",['Low-Income_Necessity Spender'],SEK,11.00,4676.909091,139273.272727
5,45,female,Netherlands,Postgraduate,Master of Law,Lawyer,18,1,4836,210402.0,"['Skiing', 'Cooking', 'Traveling']",['Wealthy_Status_Spender'],EUR,1.00,4836.000000,210402.000000
6,50,male,Spain,Secondary,High School Diploma,Construction Worker,30,1,4245,287045.0,"['Fishing', 'Football']",['Low-Income_Necessity Spender'],EUR,1.00,4245.000000,287045.000000
7,29,female,Romania,Undergraduate,Bachelor of Philosophy,Philosophy Teacher,3,1,9502,49921.0,"['Reading', 'Debating']",['Middle-Class_Value Seeker'],RON,5.00,1900.400000,9984.200000
8,32,male,Portugal,Undergraduate,Bachelor of Science in Computer Engineering,IT Specialist,6,1,2404,21039.0,"['Coding', 'Gaming', 'Cycling']",['Middle-Class_Value Seeker'],EUR,1.00,2404.000000,21039.000000
9,48,female,Germany,Postgraduate,Master of Economics,Economist,20,1,4557,191718.0,"['Traveling', 'Cooking', 'Reading']",['Wealthy_Status_Spender'],EUR,1.00,4557.000000,191718.000000


In [290]:
ai_traininig_df['spendingBehaviour'].value_counts()

Middle-Class_Value Seeker           1154
['Wealthy_Status_Spender']           296
Wealthy_Status_Spender               149
Upper-Class_Financially Stable        54
['Low-Income_Necessity Spender']      46
Low-Income_Necessity Spender          36
High-End Consumer                      8
Budget-Conscious Shopper               2
Retired                                1
Name: spendingBehaviour, dtype: int64

In [296]:

# here, since we did generated the data using several batches, there are some inconsistencies: we will fix them and 
#   devide the  spendingBehaviour into three classes
replacements = {
     "['Middle-Class_Value Seeker']" : 'Middle-Class_Value Seeker' ,
    "['Wealthy_Status_Spender']" : "Upper-Class_Financially Stable",
    "Wealthy_Status_Spender" : "Upper-Class_Financially Stable",
    "['Low-Income_Necessity Spender']" : "Low-Income_Necessity Spender",
    "High-End Consumer" :  "Upper-Class_Financially Stable",
    "Budget-Conscious Shopper" :  'Middle-Class_Value Seeker',
    "Retired" : 'Middle-Class_Value Seeker',
    "Middle-Class_Value Seeker" : "Middle-Class_Value_Seeker",
    "Upper-Class_Financially Stable" : "Upper-Class_Financially_Stable",
    "Low-Income_Necessity Spender" : "Low-Income_Necessity_Spender"
}

# Apply replacements
ai_traininig_df['spendingBehaviour'] = ai_traininig_df['spendingBehaviour'].replace(replacements)
ai_traininig_df['spendingBehaviour'].value_counts()

Middle-Class_Value_Seeker         1157
Upper-Class_Financially_Stable     507
Low-Income_Necessity_Spender        82
Name: spendingBehaviour, dtype: int64

In [297]:
ai_traininig_df .to_excel('ai_traininig_df_final.xlsx', index=False)
print("done")

done
